In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
import re
import pandas as pd
import gensim
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import nltk
import spacy
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving movie_review.csv to movie_review (1).csv


**Pre-processing des données**

In [ ]:
data = pd.read_csv('movie_review.csv')
# Prétraitement des phrases
texts = [
    [
        mot
        for mot in re.findall(r'\b[a-z][a-z]+\b', phrase.lower())
        if mot not in stopwords.words('english') and len(mot) >= 2
    ]
    for phrase in data['text'].tolist()
]

# Encodage des tags (1 pour 'pos', 0 pour 'neg')
tags = [1 if tag.strip() == 'pos' else 0 for tag in data['tag'].tolist()]

# Afficher le résultat pour la première phrase
print(f"Texte prétraité : {texts[0]} - tag : {tags[0]}")


Texte prétraité : ['films', 'adapted', 'comic', 'books', 'plenty', 'success', 'whether', 'superheroes', 'batman', 'superman', 'spawn', 'geared', 'toward', 'kids', 'casper', 'arthouse', 'crowd', 'ghost', 'world', 'never', 'really', 'comic', 'book', 'like', 'hell'] - tag : 1


**ntraînement du modèle Word2Vec**

In [ ]:
model = Word2Vec(texts, vector_size=10, window=5, min_count=1, sg=0)

**Vectorisation des reviews de movies**

In [ ]:
# Définition de la fonction review_to_vector
def review_to_vector(review, model):
    vectors = [model.wv.get_vector(word) for word in review if word in model.wv.key_to_index]

    if not vectors:
        return np.zeros(model.vector_size)

    return np.mean(vectors, axis=0)

# Appliquer la fonction review_to_vector à chaque liste
vectorized_reviews = [review_to_vector(review, model) for review in texts]

# Convertir la liste en un tableau numpy pour une manipulation plus facile
vectorized_reviews = np.array(vectorized_reviews)

# Afficher le vecteur pour la première liste
print("Vecteur pour la première liste :")
print(vectorized_reviews[0])

Vecteur pour la première liste :
[ 0.86129379 -0.92753702  0.97039992 -0.08289677  1.04790032 -0.07749916
  2.08352208  1.25491059 -1.82370579 -0.58939183]


**Division des données**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vectorized_reviews, tags, test_size=0.2, random_state=42)

**Construction d&#39;un classificateur**

In [ ]:
# Initialiser le modèle de régression logistique
#model = LogisticRegression(random_state=42)-->faible
model = RandomForestClassifier(n_estimators=200, random_state=42)

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
predictions = model.predict(X_test)

**Évaluation du modèle**

In [ ]:
# Calcul des métriques d'évaluation
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

# Afficher les résultats
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.5486711990111248
Precision: 0.5532410779315368
Recall: 0.577818347786399
F1 Score: 0.5652626878999851
